===

## Mega
Mega Combinator:
             precision    recall  f1-score   support

          0       0.86      0.75      0.80        81
          1       0.71      0.83      0.77        59

avg / total       0.80      0.79      0.79       140

===

## Doc2vec avg
Logistic Regression:
             precision    recall  f1-score   support

          0       0.85      0.75      0.80        81
          1       0.71      0.81      0.76        59

avg / total       0.79      0.78      0.78       140

Support vector machine(SVM):
              precision    recall  f1-score   support

          0       0.88      0.75      0.81        81
          1       0.72      0.86      0.78        59

avg / total       0.81      0.80      0.80       140

Gradient Boosting Classifier:
              precision    recall  f1-score   support

          0       0.85      0.78      0.81        81
          1       0.73      0.81      0.77        59

avg / total       0.80      0.79      0.79       140


## Preprocessing

In [1]:
import re 
import string
import unicodedata
import sys

STOPWORDS = [u"ấy", u"bị", u"bởi", u"cả", u"các", u"cái", u"cần", u"càng", u"chỉ", u"chiếc", u"cho", u"chứ", u"chưa", 
             u"chuyện", u"có", u"có_thể", u"cứ", u"của", u"cùng", u"cũng", u"đã", u"đang", u"đây", u"để", u"đến_nỗi", 
             u"đều", u"điều", u"do", u"đó", u"được", u"dưới", u"gì", u"khi", u"không", u"là", u"lại", u"lên", u"lúc", 
             u"mà", u"mỗi", u"một cách", u"này", u"nên", u"nếu", u"ngay", u"nhiều", u"như", u"nhưng", u"những", u"nơi", 
             u"nữa", u"phải", u"qua", u"ra", u"rằng", u"rất", u"rồi", u"sau", u"sẽ", u"so", u"sự", u"tại", u"theo", 
             u"thì", u"trên", u"trước", u"từ", u"từng", u"và", u"vẫn", u"vào", u"vậy", u"vì", u"việc", u"với", u"vừa",
             u"_num", u"wwdateww", u"wwtimeww", u"wwemailww", u"wwipww", u"wwurlww", u"wwnumberww"
            ]

tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P') and i != 45 and i!= 95)

def vi_trans_unicode(su):
    return su.translate(tbl)

def vi_strip_text2(s):
    s = re.sub(r"&amp;", "", s)
    s = vi_trans_unicode(s)
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub(r"__+", "_", s)
    s = re.sub(r"--+", "-", s)
    s = ' '.join([w if not w[0].isdigit() else u"wwNUMBERww" for w in s.strip().split()])
    return s

def vi_clean3(line):
    words = line.replace('.','').strip().split()
    words = [w.lower() for w in words]
    return ' '.join(words)

def vi_remove_stop_1char(line):
    words = line.replace('.','').strip().split()
    words = [w for w in words if w not in STOPWORDS and len(w) > 1]
    return ' '.join(words)

import requests
import codecs
import json

tok_url = "http://192.168.0.215:8081/api/v1.0/document/filter?level=0"

def get_tokens(tok_url, data):
    try:
        rq = requests.post(tok_url, data=vi_clean3(vi_strip_text2(data)).encode('utf-8'))
        if rq.content:
            tok_doc = json.loads(rq.content)['document']
        else:
            tok_doc = None
        return tok_doc
    except Exception, e:
        print e

## Loading model

## Doc2vec

In [ ]:
model = Doc2Vec.load("/home/laampt/nlp/train/model/docsent_tok_doc700K_dm_5ns_200features_2minwords_8context_e5_lowercase_nostopword_cpu_5epo_alpha_r268")

## Estimators LR, SVC, GBC

In [ ]:
joblib.load(lr_d2v_in, "/home/laampt/nlp/train/estimator/268/lr_w2v_a.pkl")
joblib.load(svc_d2v_in, "/home/laampt/nlp/train/estimator/268/svc_w2v_a.pkl")
joblib.load(gbc_d2v_in, "/home/laampt/nlp/train/estimator/268/gbc_w2v_a.pkl")
joblib.load(imputer, "/home/laampt/nlp/train/estimator/268/w2v_a_imputer.pkl")

## Pipeline

1. doc
2. clean_doc = vi_remove_stop_1char(vi_clean3(vi_strip_text2(doc)))
3. vec = makeFeatureVec(clean_doc, model, num_features)
4. vec = imputer.transform(vec)
5. mega predict

$prediction = lr\_w2v\_a.predict(vec) * 0.3 + svc\_w2v\_a.predict(vec) * 0.4 + gbc\_w2v\_a.predict(vec) * 0.3$

$prediction = 1 if (prediction[0] > 0.5) else 0$


In [ ]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float64")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs